## In this notebook we process haemofiltration data that was extracted from the Philips ICCA system to make it ready for further analysis. 

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [16]:
all_data = pd.read_csv('fresh_lab_results_haemofiltration.csv', skiprows=4, skipfooter=3, engine='python')

#### First we merge the valueNumber and valueString columns - for 'Free Form Lab' results valueNumber will be missing and valueString will contain the result.

In [111]:
def merge_values(row):
    
    val = row.valueNumber
    if np.isnan(val):
        try:
            val = float(row.valueString)
        except:
            val = np.nan
    return val

In [112]:
all_data['value'] = all_data.apply(merge_values, axis=1)

In [113]:
all_data.columns

Index([u'encounterId', u'interventionId', u'chartTime', u'storeTime',
       u'longLabel', u'attributeId', u'valueString', u'valueNumber',
       u'attribute', u'value'],
      dtype='object')

#### We then group by [encounterId, interventionId and chartTime] and for each group we take the 'value' at the latest storeTime. This is because there may be multiple storeTimes for each chartTime and we must trust the latest one by default.

In [115]:
idx = all_data.groupby(['encounterId', 'interventionId','chartTime'])['storeTime'].transform(max) == all_data['storeTime']
single_records = all_data[idx]

In [121]:
lab_results = single_records.pivot_table(index=['encounterId','chartTime'], columns='longLabel', values='value')

In [123]:
lab_results.reset_index(level=['encounterId', 'chartTime'], inplace=True)

#### We now merge the multiple HCO3 measures.

In [157]:
def merge_hco3(row):
    
    val = row[['HCO3-std']][0]
    for substitue in ['HCO3 std', 'HCO3(c)', 'HCO3-(c)']:
        if not np.isnan(val):
            break
        val = row[[substitue]][0]
    return val

In [158]:
lab_results['HCO3'] = lab_results.apply(merge_hco3, axis=1)

#### We convert the datetime format to UK standard.

In [135]:
lab_results['chartTime'] = pd.to_datetime(lab_results.chartTime)

In [140]:
lab_results['time'] = lab_results['chartTime'].dt.strftime('%d/%m/%Y %H:%M:%S')
lab_results = lab_results.drop('chartTime', axis=1)

#### And we reorder and rename the columns for clarity.

In [159]:
lab_results_clear = pd.DataFrame()
lab_results_clear['encounterId'] = lab_results['encounterId']
lab_results_clear['time'] = lab_results['time']
lab_results_clear['Sodium (blood gas)'] = lab_results['Na']
lab_results_clear['Potassium (blood gas)'] = lab_results['K']
lab_results_clear['Sodium (labs)'] = lab_results['Sodium']
lab_results_clear['Potassium (labs)'] = lab_results['Potassium']
lab_results_clear['pH'] = lab_results['pH']
lab_results_clear['Ca++'] = lab_results['Ca++']
lab_results_clear['HCO3'] = lab_results['HCO3']
lab_results_clear['Urea'] = lab_results['Urea']
lab_results_clear['Creatinine'] = lab_results['Creatinine']

In [160]:
lab_results_clear.head()

,encounterId,time,Sodium (blood gas),Potassium (blood gas),Sodium (labs),Potassium (labs),pH,Ca++,HCO3,Urea,Creatinine
0,769,01/02/2015 09:36:00,138.0,2.6,NaN,NaN,7.17,0.94,14.1,NaN,NaN
1,769,01/02/2015 09:38:00,NaN,NaN,139.0,2.9,NaN,NaN,NaN,2.6,59.0
2,769,01/02/2015 11:50:00,137.0,2.8,NaN,NaN,7.27,0.93,17.4,NaN,NaN
3,769,01/02/2015 12:15:00,NaN,NaN,141.0,3.3,NaN,NaN,NaN,2.8,53.0
4,769,01/02/2015 12:55:00,136.0,3.1,NaN,NaN,7.28,0.98,17.5,NaN,NaN


#### Finally we save the processed data as a csv file for portability.

In [161]:
lab_results_clear.to_csv('haemofiltration_labresults_processed.csv')

-----------------------------------------------------------------------------------
### We will  now follow a similar procedure to process the flowsheet variables relating to haemofiltration.

In [170]:
all_data = pd.read_csv('fresh_pt_assessment_haemofiltration.rpt', skiprows=2, skipfooter=3, engine='python', delimiter='\t')

#### First we merge the valueNumber and valueString columns, putting everything intpo string format.

In [180]:
def merge_values(row):
    
    val = row.valueString
    if not type(val) is str:
        try:
            val = str(row.valueNumber)
        except:
            val = np.nan
    return val

In [181]:
all_data['value'] = all_data.apply(merge_values, axis=1)

In [182]:
all_data.columns

Index([u'encounterId', u'interventionId', u'chartTime', u'storeTime',
       u'longLabel', u'attributeId', u'valueString', u'valueNumber',
       u'attribute', u'value'],
      dtype='object')

#### We then group by [encounterId, interventionId and chartTime] and for each group we take the 'value' at the latest storeTime. This is because there may be multiple storeTimes for each chartTime and we must trust the latest one by default.

In [211]:
idx = all_data.groupby(['encounterId', 'interventionId','chartTime'])['storeTime'].transform(max) == all_data['storeTime']
single_records = all_data[idx]

In [212]:
flowsheet = single_records.pivot_table(index=['encounterId','chartTime'], columns='longLabel', values='value', aggfunc=lambda x: ' '.join(x))

In [213]:
flowsheet.reset_index(level=['encounterId', 'chartTime'], inplace=True)

In [214]:
flowsheet['encounterId'] = pd.to_numeric(flowsheet.encounterId)
flowsheet['chartTime'] = pd.to_datetime(flowsheet.chartTime)
flowsheet.sort_values(by=['encounterId', 'chartTime'], ascending=True, inplace=True)

#### We convert the datetime format to UK standard.

In [217]:
flowsheet['chartTime'] = pd.to_datetime(flowsheet.chartTime)

In [218]:
flowsheet['chartTime'] = flowsheet['chartTime'].dt.strftime('%d/%m/%Y %H:%M:%S')

In [224]:
cols = flowsheet.columns.values
cols[1] = 'time'
flowsheet.columns = cols

In [225]:
flowsheet.head()

,encounterId,time,Blood Flow Rate,Calcium Chloride Dose Adjustment,Calcium Dose,Citrate Dose,Citrate Dose Adjustment,Exchange Rate,Filter Set,Filter in Use,First Checker,Fluid Removed,Pre-Filter Pressure,Reason for Filter Loss,Return (Venous) Pressure,Therapy Run Time,Therapy Type
100965,794,03/02/2015 22:00:00,100.0,None,None,None,None,25.0,ST150,None,None,0.0,None,None,56.0,None,CVVHDF
100966,794,03/02/2015 23:00:00,200.0,None,None,None,None,25.0,ST150,None,None,100.0,None,None,28.0,None,CVVHDF
100967,794,04/02/2015 01:00:00,200.0,None,None,None,None,25.0,ST150,None,None,100.0,None,None,130.0,None,CVVHDF
100968,794,04/02/2015 02:00:00,200.0,None,None,None,None,25.0,ST150,None,None,200.0,None,None,125.0,None,CvvHDF
100969,794,04/02/2015 03:00:00,200.0,None,None,None,None,25.0,St150,None,None,200.0,None,None,140.0,None,CVVHDF


#### And we save the data as csv.

In [226]:
flowsheet.to_csv('haemofiltration_ptassessment_processed.csv')

-----------------------------------------------------------------------------------
### Finally we process the patient summary data.

In [18]:
all_data = pd.read_csv('fresh_patient_summary_haemofiltration.rpt', skiprows=0, skipfooter=4, engine='python', delimiter='\t')

#### Here we simply convert the datetime format and the column names for clarity.

In [20]:
all_data['inTIME'] = pd.to_datetime(all_data.inTIME)
all_data['outTime'] = pd.to_datetime(all_data.outTime)

In [21]:
all_data['outTime'] = all_data['outTime'].dt.strftime('%d/%m/%Y %H:%M:%S')
all_data['inTIME'] = all_data['inTIME'].dt.strftime('%d/%m/%Y %H:%M:%S')

In [23]:
cols = all_data.columns.values

In [25]:
cols[0] = 'encounterId'
cols[1] = 'inTime'
all_data.columns = cols

In [26]:
all_data.to_csv('haemofiltration_patient_summary_processed.csv')

In [28]:
## We do not display the data here because it contains identifiable patient info.
##all_data.head()